# <b>Python for Data Analysis</b>
# 10.  Data Aggregation and Group Operations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision = 4, suppress = True)
from pandas import Series, DataFrame
%matplotlib inline

## 10.1 How to Think About Group Operations

In [2]:
df = pd.DataFrame({"key1":  ["a", "a", None, "b", "b", "a", None],
                   "key2":  pd.Series([1, 2, 1, 2, 1, None, 1], dtype = "Int64"),
                   "data1": np.random.standard_normal(7),
                   "data2": np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-1.326374,0.278792
1,a,2,1.602728,0.274606
2,None,1,0.328159,0.128529
3,b,2,0.018278,1.016007
4,b,1,-0.451523,-0.730365
5,a,<NA>,1.071761,-0.580650
6,None,1,0.203720,2.446457


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()

key1
a    0.449372
b   -0.216623
Name: data1, dtype: float64

In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -1.326374
      2       1.602728
b     1      -0.451523
      2       0.018278
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,1,2
key1,,
a,-1.326374,1.602728
b,-0.451523,0.018278


In [7]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [8]:
states

array(['OH', 'CA', 'CA', 'OH', 'OH', 'CA', 'OH'], dtype='<U2')

In [9]:
years

[2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [10]:
df["data1"].groupby([states, years]).mean()

CA  2005    1.337245
    2006    0.328159
OH  2005   -0.654048
    2006   -0.123902
Name: data1, dtype: float64

In [11]:
df

,key1,key2,data1,data2
0,a,1,-1.326374,0.278792
1,a,2,1.602728,0.274606
2,None,1,0.328159,0.128529
3,b,2,0.018278,1.016007
4,b,1,-0.451523,-0.730365
5,a,<NA>,1.071761,-0.580650
6,None,1,0.203720,2.446457


In [12]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.449372,-0.009084
b,1.5,-0.216623,0.142821


In [13]:
df.groupby("key2").mean(numeric_only = True)

,data1,data2
key2,,
1,-0.311504,0.530853
2,0.810503,0.645307


In [14]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -1.326374  0.278792
     2     1.602728  0.274606
b    1    -0.451523 -0.730365
     2     0.018278  1.016007

In [15]:
df.groupby(["key1", "key2"], dropna = False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [16]:
df.groupby("key1", dropna = False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [17]:
df.groupby(["key1", "key2"], dropna = False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [18]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

In [19]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

In [20]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

### Iterating over Groups

In [21]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -1.326374  0.278792
1    a     2  1.602728  0.274606
5    a  <NA>  1.071761 -0.580650
b
  key1  key2     data1     data2
3    b     2  0.018278  1.016007
4    b     1 -0.451523 -0.730365


In [22]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1 -1.326374  0.278792
('a', 2)
  key1  key2     data1     data2
1    a     2  1.602728  0.274606
('b', 1)
  key1  key2     data1     data2
4    b     1 -0.451523 -0.730365
('b', 2)
  key1  key2     data1     data2
3    b     2  0.018278  1.016007


In [23]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.018278,1.016007
4,b,1,-0.451523,-0.730365


In [24]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"},
                     axis = "columns")

In [25]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -1.326374  0.278792
1  1.602728  0.274606
2  0.328159  0.128529
3  0.018278  1.016007
4 -0.451523 -0.730365
5  1.071761 -0.580650
6  0.203720  2.446457
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


### Selecting a Column or Subset of Columns

In [26]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [27]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [28]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.278792
     2     0.274606
b    1    -0.730365
     2     1.016007

In [29]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [30]:
s_grouped.mean()

key1  key2
a     1       0.278792
      2       0.274606
b     1      -0.730365
      2       1.016007
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [31]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns = ["a", "b", "c", "d", "e"],
                      index = ["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,-0.061999,-0.609897,-1.033106,-0.286834,1.261808
Steve,-1.010808,2.166611,-0.667524,0.931679,0.357416
Wanda,0.603449,-0.108783,-0.285272,-0.727248,-2.054913
Jill,-0.857377,2.056508,1.295811,0.049886,-1.236229
Trey,0.773347,-0.854422,1.256283,-0.388508,0.410596


In [32]:
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-0.061999,-0.609897,-1.033106,-0.286834,1.261808
Steve,-1.010808,2.166611,-0.667524,0.931679,0.357416
Wanda,0.603449,NaN,NaN,-0.727248,-2.054913
Jill,-0.857377,2.056508,1.295811,0.049886,-1.236229
Trey,0.773347,-0.854422,1.256283,-0.388508,0.410596


In [33]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [34]:
by_column = people.groupby(mapping, axis = "columns")
by_column.sum()

,blue,red
Joe,-1.319940,0.589913
Steve,0.264154,1.513219
Wanda,-0.727248,-1.451464
Jill,1.345697,-0.037098
Trey,0.867775,0.329521


In [35]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [36]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [37]:
people

,a,b,c,d,e
Joe,-0.061999,-0.609897,-1.033106,-0.286834,1.261808
Steve,-1.010808,2.166611,-0.667524,0.931679,0.357416
Wanda,0.603449,NaN,NaN,-0.727248,-2.054913
Jill,-0.857377,2.056508,1.295811,0.049886,-1.236229
Trey,0.773347,-0.854422,1.256283,-0.388508,0.410596


In [38]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.061999,-0.609897,-1.033106,-0.286834,1.261808
4,-0.084030,1.202085,2.552094,-0.338622,-0.825632
5,-0.407359,2.166611,-0.667524,0.204430,-1.697497


In [39]:
key_list = ["one", "one", "one", "two", "two"]

In [40]:
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-0.061999,-0.609897,-1.033106,-0.286834,1.261808
4,two,-0.857377,-0.854422,1.256283,-0.388508,-1.236229
5,one,-1.010808,2.166611,-0.667524,-0.727248,-2.054913


### Grouping by Index Levels

In [41]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names = ["cty", "tenor"])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [42]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), 
                       columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -1.262044  1.148098  0.026516 -1.540757 -1.448658
1     -1.174919 -1.588653  1.335610 -0.843189 -0.383783
2      0.930746  1.986062  0.100765 -0.914000 -0.573532
3      0.500986 -2.587112 -1.697831 -1.809703 -1.896388

In [43]:
hier_df.groupby(level = "cty", axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [44]:
df

,key1,key2,data1,data2
0,a,1,-1.326374,0.278792
1,a,2,1.602728,0.274606
2,None,1,0.328159,0.128529
3,b,2,0.018278,1.016007
4,b,1,-0.451523,-0.730365
5,a,<NA>,1.071761,-0.580650
6,None,1,0.203720,2.446457


In [46]:
grouped = df.groupby("key1")
grouped

In [49]:
grouped["data1"].nsmallest(2)

key1   
a     0   -1.326374
      5    1.071761
b     4   -0.451523
      3    0.018278
Name: data1, dtype: float64

In [50]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [51]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,2.929102,0.859443
b,1,0.469801,1.746372


In [52]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.449372  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.216623  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     1.337245  1.602728   3.0 -0.009084  0.494995 -0.580650 -0.153022   
b    -0.099172  0.018278   2.0  0.142821  1.234871 -0.730365 -0.293772   

                                    
           50%       75%       max  
key1                                
a     0.274606  0.276699  0.278792  
b     0.142821  0.579414  1.016007  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [56]:
tips = pd.read_csv("examples/tips.csv")

In [57]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [60]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [61]:
grouped = tips.groupby(["day", "smoker"])

In [62]:
grouped_pct = grouped["tip_pct"]

In [63]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [66]:
tips.groupby(["day", "smoker"])["tip_pct"].agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [67]:
tips.groupby(["day", "smoker"])["tip_pct"].mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [69]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [72]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)]) # (column name, function)

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [73]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")]) # (column name, function)

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [74]:
grouped_pct.agg(["mean", "std", ("Max/min diff.", peak_to_peak)])

mean       std  Max/min diff.
day  smoker                                   
Fri  No      0.151650  0.028123       0.067349
     Yes     0.174783  0.051293       0.159925
Sat  No      0.158048  0.039767       0.235193
     Yes     0.147906  0.061375       0.290095
Sun  No      0.160113  0.042347       0.193226
     Yes     0.187250  0.154134       0.644685
Thur No      0.160298  0.038774       0.193350
     Yes     0.163863  0.039389       0.151240

In [75]:
functions = ["count", "mean", "max"]

In [76]:
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [77]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [78]:
ftuples = [("Average", "mean"), ("Variance", np.var)]

In [79]:
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [80]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [83]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
             "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [84]:
grouped = tips.groupby(["day", "smoker"], as_index = False)
grouped.mean(numeric_only = True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [86]:
result_test = grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
                           "size": "sum"})
result_test.reset_index()

index   day smoker   tip_pct                               size
                           min       max      mean       std  sum
0     0   Fri     No  0.120385  0.187735  0.151650  0.028123    9
1     1   Fri    Yes  0.103555  0.263480  0.174783  0.051293   31
2     2   Sat     No  0.056797  0.291990  0.158048  0.039767  115
3     3   Sat    Yes  0.035638  0.325733  0.147906  0.061375  104
4     4   Sun     No  0.059447  0.252672  0.160113  0.042347  167
5     5   Sun    Yes  0.065660  0.710345  0.187250  0.154134   49
6     6  Thur     No  0.072961  0.266312  0.160298  0.038774  112
7     7  Thur    Yes  0.090014  0.241255  0.163863  0.039389   40

## 10.3 Apply: General split-apply-combine

In [90]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [87]:
def top(df, n = 5, column = "tip_pct"):
    return df.sort_values(column, ascending = False)[:n]

In [88]:
top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [98]:
tips.sort_values("tip", ascending = False)[:5]

,total_bill,tip,smoker,day,time,size,tip_pct
170,50.81,10.00,Yes,Sat,Dinner,3,0.196812
212,48.33,9.00,No,Sat,Dinner,4,0.186220
23,39.42,7.58,No,Sat,Dinner,4,0.192288
59,48.27,6.73,No,Sat,Dinner,4,0.139424
141,34.30,6.70,No,Thur,Lunch,6,0.195335


In [99]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [100]:
tips.groupby(["smoker", "day"]).apply(top, n = 1, column = "total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [101]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [102]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

This code is alternative to <code>.describe()</code>:

In [103]:
# def f(group):
#    return group.describe()

# grouped.apply(f)

### Suppressing the Group Keys

In [104]:
tips.groupby("smoker", group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [105]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

### Quantile and Bucket Analysis

In [106]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-1.034412,0.206537
1,0.336484,-0.517357
2,0.788918,1.442342
3,-0.276378,-1.124211
4,-0.430930,1.076233


In [107]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0    (-1.657, -0.204]
1     (-0.204, 1.249]
2     (-0.204, 1.249]
3    (-1.657, -0.204]
4    (-1.657, -0.204]
5     (-0.204, 1.249]
6     (-0.204, 1.249]
7     (-0.204, 1.249]
8     (-0.204, 1.249]
9     (-0.204, 1.249]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.116, -1.657] < (-1.657, -0.204] < (-0.204, 1.249] < (1.249, 2.702]]

In [108]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
         "count": group.count(), "mean": group.mean()})

In [109]:
grouped = frame.groupby(quartiles)

In [110]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.116, -1.657] data1 -3.110442 -1.668743     46 -2.015317
                 data2 -2.773428  2.618239     46 -0.221723
(-1.657, -0.204] data1 -1.636551 -0.205327    379 -0.779140
                 data2 -2.686757  3.427729    379  0.071861
(-0.204, 1.249]  data1 -0.203746  1.246136    488  0.466537
                 data2 -3.061205  3.220421    488 -0.090433
(1.249, 2.702]   data1  1.254995  2.702021     87  1.742889
                 data2 -2.776807  1.871383     87  0.085054

In [113]:
grouped.agg(["min", "max", "count", "mean"]).unpivot()

AttributeError: 'DataFrame' object has no attribute 'unpivot'

In [117]:
quartiles_samp = pd.qcut(frame["data1"], 4)
quartiles_samp.head()

0    (-3.1109999999999998, -0.678]
1                  (0.0103, 0.657]
2                   (0.657, 2.702]
3                 (-0.678, 0.0103]
4                 (-0.678, 0.0103]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.1109999999999998, -0.678] < (-0.678, 0.0103] < (0.0103, 0.657] < (0.657, 2.702]]

In [115]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.110442 -0.683723    250 -1.256785
      data2 -2.773428  3.427729    250  0.033180
1     data1 -0.676049  0.009696    250 -0.322291
      data2 -2.924796  2.645530    250 -0.029843
2     data1  0.010913  0.656309    250  0.325840
      data2 -3.061205  3.220421    250 -0.042331
3     data1  0.657830  2.702021    250  1.218447
      data2 -2.776807  2.205774    250 -0.039787

In [117]:
quartiles_samp = pd.qcut(frame["data1"], 4)
quartiles_samp.head()

0    (-3.1109999999999998, -0.678]
1                  (0.0103, 0.657]
2                   (0.657, 2.702]
3                 (-0.678, 0.0103]
4                 (-0.678, 0.0103]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.1109999999999998, -0.678] < (-0.678, 0.0103] < (0.0103, 0.657] < (0.657, 2.702]]

### Example: Filling Missing Values with Group-Specific Values

In [120]:
s = pd.Series(np.random.standard_normal(6))
s

0    1.295789
1   -0.329187
2   -0.126364
3   -1.230774
4    0.169885
5   -1.989059
dtype: float64

In [121]:
s[::2] = np.nan
s

0         NaN
1   -0.329187
2         NaN
3   -1.230774
4         NaN
5   -1.989059
dtype: float64

In [122]:
s.fillna(s.mean())

0   -1.183007
1   -0.329187
2   -1.183007
3   -1.230774
4   -1.183007
5   -1.989059
dtype: float64

In [123]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]

group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]

In [124]:
data = pd.Series(np.random.standard_normal(8), index = states)
data

Ohio         -0.223340
New York      0.692471
Vermont       0.175822
Florida      -0.209991
Oregon        0.229805
Nevada       -0.874578
California   -1.186803
Idaho         0.128550
dtype: float64

In [126]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.223340
New York      0.692471
Vermont            NaN
Florida      -0.209991
Oregon        0.229805
Nevada             NaN
California   -1.186803
Idaho              NaN
dtype: float64

In [129]:
data.groupby(group_key).mean()

East    0.086380
West   -0.478499
dtype: float64

In [129]:
data.groupby(group_key).mean()

East    0.086380
West   -0.478499
dtype: float64

In [129]:
data.groupby(group_key).mean()

East    0.086380
West   -0.478499
dtype: float64

In [130]:
def fill_mean(group):
    return group.fillna(group.mean())

In [131]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.223340
New York      0.692471
Vermont       0.086380
Florida      -0.209991
Oregon        0.229805
Nevada       -0.478499
California   -1.186803
Idaho        -0.478499
dtype: float64

In [132]:
fill_values = {"East": 0.5, "West": -1}

In [133]:
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [134]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.223340
New York      0.692471
Vermont       0.500000
Florida      -0.209991
Oregon        0.229805
Nevada       -1.000000
California   -1.186803
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [140]:
suits = list("HSCD")
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + list("JKQ")
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index = cards)

In [164]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
       ..
9D      9
10D    10
JD     10
KD     10
QD     10
Length: 52, dtype: int64

In [141]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [158]:
def draw(deck, n = 5):
    return deck.sample(n) # return a n-number of values from a df

In [159]:
draw(deck)

JS    10
QD    10
5D     5
8D     8
3H     3
dtype: int64

In [160]:
deck.sample(10)

JH    10
4C     4
9S     9
AH     1
AS     1
QS    10
3S     3
9C     9
JC    10
8H     8
dtype: int64

In [161]:
def get_suit(card):
    # last letter is suit
    return card[-1]

In [162]:
deck.groupby(get_suit).apply(draw, n = 2)

C  7C     7
   2C     2
D  4D     4
   JD    10
H  9H     9
   4H     4
S  KS    10
   5S     5
dtype: int64

In [163]:
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

7C      7
8C      8
5D      5
4D      4
10H    10
5H      5
10S    10
6S      6
dtype: int64

### Example: Group Weighted Average and Correlation

In [165]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size = 8)})
df

,category,data,weights
0,a,-0.930173,0.887922
1,a,-0.411332,0.679374
2,a,0.411038,0.250076
3,a,0.347251,0.077935
4,b,0.993339,0.755540
5,b,-0.323572,0.696881
6,b,-1.136582,0.796446
7,b,0.559328,0.292873


In [166]:
grouped = df.groupby("category")

In [167]:
def get_wavg(group):
    return np.average(group["data"], weights = group["weights"])

In [168]:
grouped.apply(get_wavg)

category
a   -0.514701
b   -0.085138
dtype: float64

In [180]:
get_wavg(df[df.category == "b"])

-0.08513751474271525

In [180]:
get_wavg(df[df.category == "b"])

-0.08513751474271525

In [180]:
get_wavg(df[df.category == "b"])

-0.08513751474271525

In [182]:
close_px = pd.read_csv("examples/stock_px.csv",
                       parse_dates = True,
                       index_col = 0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [184]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [184]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58
